In [2]:
import numpy as np
import pandas as pd

In [3]:
iris_df=pd.read_csv('./iris.csv')

groups=iris_df.groupby('variety')

testing_set=groups.sample(1,random_state=100)
training_set=[]

training_set=iris_df[~iris_df.isin(testing_set).all(axis=1)]

display(training_set)
display(testing_set)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


,sepal.length,sepal.width,petal.length,petal.width,variety
6,4.6,3.4,1.4,0.3,Setosa
66,5.6,3.0,4.5,1.5,Versicolor
129,7.2,3.0,5.8,1.6,Virginica


In [4]:
mean_vectors=groups.mean()
display(mean_vectors)

,sepal.length,sepal.width,petal.length,petal.width
variety,,,,
Setosa,5.006,3.428,1.462,0.246
Versicolor,5.936,2.770,4.260,1.326
Virginica,6.588,2.974,5.552,2.026


In [5]:
def calculate_covariance(x,y,mean_x,mean_y):
    return ((np.sum((x-mean_x)*(y-mean_y)))/len(x))

def get_covariance_matrix(training_set,mean_vector):
    covariance=[]
    for i in training_set:
        for j in training_set:
            covariance.append(calculate_covariance(training_set[i].tolist(),training_set[j].tolist(),mean_vector[i],mean_vector[j]))
    return np.array(covariance).reshape(len(mean_vector),len(mean_vector))

def mahalanobis_distance(x,mean_x,cov_mat_inv):
    return (np.dot(np.dot((x-mean_x).transpose(),cov_mat_inv),(x-mean_x)))
    

def classify(training_set,testing_set,mean_vectors):
    result_set=testing_set.copy()
    testing_set=testing_set.drop(['variety'],axis=1)
    cov_mat_inv={}
    for index in mean_vectors.index:
        cov_mat_inv[index]=np.linalg.inv((get_covariance_matrix(training_set.get_group((index,)).drop(['variety'],axis=1),mean_vectors.loc[index])))

    predicted_class=[]
    print("Mahalanobis Distances:")
    for point in testing_set.values:
        dist=pd.Series()
        for index in mean_vectors.index:
            dist[index]=mahalanobis_distance(point,mean_vectors.loc[index],cov_mat_inv[index])
        predicted_class.append(dist.idxmin())
        print(dist)
    result_set['predicted class']=predicted_class

    return result_set

In [16]:
result_set=classify(training_set.groupby(['variety']),testing_set,mean_vectors)
display(result_set)

Mahalanobis Distances:
Setosa          3.681462
Versicolor     96.274979
Virginica     159.077602
dtype: float64
Setosa        379.844925
Versicolor      3.801569
Virginica       8.956824
dtype: float64
Setosa        718.650350
Versicolor     14.783018
Virginica       5.607748
dtype: float64


,sepal.length,sepal.width,petal.length,petal.width,variety,predicted class
6,4.6,3.4,1.4,0.3,Setosa,Setosa
66,5.6,3.0,4.5,1.5,Versicolor,Versicolor
129,7.2,3.0,5.8,1.6,Virginica,Virginica


In [19]:
temp=pd.DataFrame([
    [1,2],
    [3,5],
    [4,3],
    [5,6],
    [7,5],
    [6,2],
    [9,4],
    [10,1],
    [12,3],
    [13,6]])
cov_mat=get_covariance_matrix(temp,temp.mean())
display(cov_mat)
scatter_mat=(1/10)*cov_mat
display(scatter_mat)

array([[14.  ,  0.7 ],
       [ 0.7 ,  2.81]])

array([[1.4  , 0.07 ],
       [0.07 , 0.281]])